In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poetry/Kanye_West.txt
/kaggle/input/poetry/johnny-cash.txt
/kaggle/input/poetry/kanye-west.txt
/kaggle/input/poetry/bruno-mars.txt
/kaggle/input/poetry/dickinson.txt
/kaggle/input/poetry/amy-winehouse.txt
/kaggle/input/poetry/blink-182.txt
/kaggle/input/poetry/paul-simon.txt
/kaggle/input/poetry/patti-smith.txt
/kaggle/input/poetry/bieber.txt
/kaggle/input/poetry/disney.txt
/kaggle/input/poetry/jimi-hendrix.txt
/kaggle/input/poetry/lin-manuel-miranda.txt
/kaggle/input/poetry/adele.txt
/kaggle/input/poetry/dj-khaled.txt
/kaggle/input/poetry/beatles.txt
/kaggle/input/poetry/r-kelly.txt
/kaggle/input/poetry/lady-gaga.txt
/kaggle/input/poetry/radiohead.txt
/kaggle/input/poetry/britney-spears.txt
/kaggle/input/poetry/alicia-keys.txt
/kaggle/input/poetry/rihanna.txt
/kaggle/input/poetry/joni-mitchell.txt
/kaggle/input/poetry/dolly-parton.txt
/kaggle/input/poetry/drake.txt
/kaggle/input/poetry/Lil_Wayne.txt
/kaggle/input/poetry/notorious_big.txt
/kaggle/input/poetry/eminem.txt
/

In [185]:
import glob
import random
import textwrap

order = 1
word_probabilities = {}

dataset_folder = "/kaggle/input/poetry/"

def markov(dataset_folder):
    global word_probabilities
    word_probabilities = {}

    file_paths = glob.glob(dataset_folder + "*.txt")
    for file_path in file_paths:
        with open(file_path, "r") as dataset_file:
            poems = dataset_file.read().split("-")
            for poem in poems:
                lines = poem.strip().split("\n")
                previouss = []
                for line in lines:
                    words = line.strip().split()

                    if len(words) < order:
                        continue
                    for i in range(len(words) - order):
                        current_word = words[i]
                        following_word = words[i + order]
                        if current_word not in word_probabilities:
                            word_probabilities[current_word] = {}
                        if following_word not in word_probabilities[current_word]:
                            word_probabilities[current_word][following_word] = 0
                        word_probabilities[current_word][following_word] += 1

def generate(start, length):
    poem = [start]
    current_word = start
    for _ in range(length - 1):
        if current_word in word_probabilities:
            next_word = random.choices(
                list(word_probabilities[current_word].keys()),
                list(word_probabilities[current_word].values()))[0]
        else:
            break

        poem.append(next_word)
        current_word = next_word

    lines = []
    line = ""
    word_count = 0
    probabilities = []
    for word in poem:
        line += word + " "
        word_count += 1

        if word_count % 5 == 0:
            lines.append(line)
            line = ""

        if word_count > 1:
            previous = poem[word_count - 2]
            probability = word_probabilities[previous][word] / sum(word_probabilities[previous].values())
            probabilities.append((previous, word, probability))

    print(probabilities)
    return "\n".join(lines), probabilities

markov(dataset_folder)
Input = input("Please enter the first word: ")
generated, _ = generate(Input, 50)
wrapped_poem = textwrap.fill(generated, width=40)
print("\n"+wrapped_poem)

Please enter the first word: 
 Woman


[('Woman', 'hold', 0.42105263157894735), ('hold', 'you', 0.17556346381969157), ('you', 'in', 0.010171471770729528), ('in', 'cars', 0.000550244170850815), ('cars', 'on', 0.02666666666666667), ('on', 'my', 0.10303921568627451), ('my', 'mom,', 0.0002337540906965872), ('mom,', "there's", 0.2), ("there's", 'forty', 0.0038240917782026767), ('forty', 'on', 0.02857142857142857), ('on', 'fire', 0.01431372549019608), ('fire', 'in', 0.037037037037037035), ('in', 'noble', 6.878052135635188e-05), ('noble', 'band,', 0.16666666666666666), ('band,', 'I', 0.1111111111111111), ('I', 'am,', 0.001833467489491101), ('am,', 'yes', 0.075), ('yes', 'she', 0.015228426395939087), ('she', 'knows', 0.012850862360500507), ('knows', 'the', 0.04201680672268908), ('the', 'motherfucking', 0.00020269357236160534), ('motherfucking', 'people', 0.02564102564102564), ('people', 'confused,', 0.0013698630136986301), ('confused,', 'you', 0.14285714285714285), ('you', 'baby)', 0.0004467200439847428), ('baby)', 'Baby', 0.0625),

In [187]:
import os
import textwrap
from pgmpy.models import BayesianModel

def bayesianNetwork(poems, n_gram=2):
    edges = {}
    model = BayesianModel()
    for poem in poems:
        poem_tokens = poem.split()
        
        for i in range(len(poem_tokens) - n_gram):
            current = tuple(poem_tokens[i:i+n_gram])
            next = poem_tokens[i + n_gram]
            
            edges.setdefault(current, {}).setdefault(next, 0)
            edges[current][next] += 1
    
    for current, all_next in edges.items():
        for next, count in all_next.items():
            model.add_edge(current, next, weight=count)
    
    return model

def generate(bayesian_network, words=50, start=('', '')):
    word_counter = 0
    position = start
    poem = f"{position[0]} {position[1]}"
    
    while word_counter < words:
        neighbors = list(bayesian_network.neighbors(position))
        if not neighbors:
            break 
        next = random.choices(neighbors, weights=[bayesian_network.get_edge_data(position, neighbor)['weight'] for neighbor in neighbors])[0]
        poem += f" {next}"
        position = tuple(list(position[1:]) + [next])
        n += 1

    return poem

bayesian_network = bayesianNetwork(poems)

first = input("Please enter the first word: ")
second = input("Please enter the second word: \n")

generated = generate(bayesian_network, words=50, start=(first, second))
wrapped_poem = textwrap.fill(generated, width=40)
print(wrapped_poem)

Please enter the first word:  i
Please enter the second word: 
 love


i love her down memory lane You know I
wish there were other guys Summer boy
Lets get lost with you You'll lose your
mind right Like two ships passing spied
a cheek Of a delivered syllable, 'T
would crumble with the icons with Nikes
on I wish that I got some


In [7]:
pip install pomegranate

  Obtaining dependency information for pomegranate from https://files.pythonhosted.org/packages/a7/6f/523c38b03df252cb5ea1e17c71e27ff7e0ca908194e6115280149b8063e8/pomegranate-1.0.3-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 3.7 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48766 sha256=a6a8ef44c3b76b9115a757f8fe62031ba5998fd35452643737da1b67668b11f1
  Stored in directory: /root/.cache/pip/wheels/df/d8/f9/4d62b7272bff772a126a52e507212c2fd33c0b8416d9389665
Successfully built apricot-select
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pgmpy

Note: you may need to restart the kernel to use updated packages.
